<a href="https://colab.research.google.com/github/PauSanchoBeneyto/Mercadona_Pontia/blob/main/API_MERCADONAv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests as req
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup as bs

In [2]:
#categoría
categorias='https://tienda.mercadona.es/api/categories/'
#tipo_categoria
url='https://tienda.mercadona.es/api/categories/112/'
#productos
productos='https://tienda.mercadona.es/api/products/'

In [3]:
#para ver el desglose en https://jsonviewer.stack.hu/
r_cat = req.get(categorias)
display(r_cat.text)

'{"next":null,"count":26,"results":[{"id":12,"name":"Aceite, especias y salsas","order":7,"layout":2,"published":true,"categories":[{"id":112,"name":"Aceite, vinagre y sal","order":7,"layout":1,"published":true,"is_extended":false},{"id":115,"name":"Especias","order":7,"layout":1,"published":true,"is_extended":false},{"id":116,"name":"Mayonesa, ketchup y mostaza","order":7,"layout":1,"published":true,"is_extended":false},{"id":117,"name":"Otras salsas","order":7,"layout":1,"published":true,"is_extended":false}],"is_extended":false},{"id":18,"name":"Agua y refrescos","order":8,"layout":2,"published":true,"categories":[{"id":156,"name":"Agua","order":8,"layout":1,"published":true,"is_extended":false},{"id":163,"name":"Isotónico y energético","order":8,"layout":1,"published":true,"is_extended":false},{"id":158,"name":"Refresco de cola","order":8,"layout":1,"published":true,"is_extended":false},{"id":159,"name":"Refresco de naranja y de limón","order":8,"layout":1,"published":true,"is_exte

In [4]:
r_url = req.get(url)
display(r_url.text)

'{"id":112,"name":"Aceite, vinagre y sal","order":7,"layout":1,"published":true,"categories":[{"id":420,"name":"Aceite de oliva","order":7,"layout":2,"products":[{"id":"4240","slug":"aceite-oliva-04o-hacendado-botella","limit":999,"badges":{"is_water":false,"requires_age_check":false},"packaging":"Botella","published":true,"share_url":"https://tienda.mercadona.es/product/4240/aceite-oliva-04o-hacendado-botella","thumbnail":"https://prod-mercadona.imgix.net/images/1a84cd7052b68873985104ac24b87043.jpg?fit=crop&h=300&w=300","categories":[{"id":12,"name":"Aceite, especias y salsas","level":0,"order":7}],"display_name":"Aceite de oliva 0,4º Hacendado","unavailable_from":null,"price_instructions":{"iva":5,"is_new":false,"is_pack":false,"pack_size":null,"unit_name":null,"unit_size":1.0,"bulk_price":"8.00","unit_price":"8.00","approx_size":false,"size_format":"l","total_units":null,"unit_selector":true,"bunch_selector":false,"drained_weight":null,"selling_method":0,"price_decreased":false,"ref

In [5]:
cat=r_cat.json()
url=r_url.json()
#prod =req.get(productos).json()

In [6]:
df_cat=pd.json_normalize(cat)
df_url=pd.json_normalize(url)

In [7]:
df_cat = pd.DataFrame(columns=['ID_cat', 'Nombre_categoria'])
df_pro = pd.DataFrame(columns=['ID_cat','subcategoria','ID_subcat','categoria_producto','ID_pro', 'Nombre', 'Precio'])
lista_subcategorias=[]

In [8]:
url = 'https://tienda.mercadona.es/api/categories/'
response = req.get(url)
lista_categorias = []
if response.status_code == 200:
    categorias = response.json()
    if 'results' in categorias:
        for categoria in categorias['results']:
            #print(f"ID {categoria['id']}: {categoria['name']}")
            cat=categoria['id']
            #OBTENEMOS EL LISTA DE  TODAS LAS CATEGORIAS PARA OBTENER LOS PRODUCTOS
            lista_categorias.append(cat)
            #Guardamos nuestra lista de productos con el id para poder concatenar en un futuro con los productos
            id=categoria['id']
            name_cat=categoria['name']
            catego = {'ID_cat': [id], 'Nombre_categoria': [name_cat]}
            df_2 = pd.DataFrame(catego)
            df_cat = pd.concat([df_cat,df_2])

In [9]:
df_pro = pd.DataFrame()
df=pd.DataFrame()

In [10]:
url = 'https://tienda.mercadona.es/api/categories/'
response = req.get(url)
lista_subcategorias = []

if response.status_code == 200:
    subcategorias = response.json()
    if 'results' in subcategorias:
        for categoria in subcategorias['results']:
            if 'categories' in categoria:
                for subcategoria in categoria['categories']:
                    id_sub = subcategoria['id']
                    name_subcat = subcategoria['name']
                    catego = {'ID_subcat': id_sub, 'subcategoria': name_subcat}
                    lista_subcategorias.append(id_sub)

# Crear DataFrame de las subcategorías
df_subcat = pd.DataFrame(lista_subcategorias)

In [11]:
for i in lista_subcategorias:
  url = "https://tienda.mercadona.es/api/categories/{}/"
  url=url.format(i)
  response = req.get(url)
  if response.status_code == 200:
      categoria = response.json()
      if 'categories' in categoria:
          for cat_info in categoria['categories']:
              if 'products' in cat_info:
                  for product in cat_info['products']:
                      product_id = product['id']
                      product_name = product['display_name']
                      product_price = product['price_instructions']['unit_price']
                      precio_KG_L = product['price_instructions']['bulk_price']
                      unidad_medida = product['price_instructions']['size_format']
                      i = categoria['id']
                      c = categoria['name']
                      id_2=cat_info['id']
                      pro=cat_info['name']
                      datos = {'ID_cat': [i], 'subcategoria': [c], 'ID_subcat':[id_2],'categoria_producto':[pro],'ID_pro': [product_id],'Nombre':[product_name],'Precio':[product_price],'Precio_unidad':[precio_KG_L],'Unidad_medida':[unidad_medida]}
                      df = pd.DataFrame(datos)
                      df_pro = pd.concat([df_pro,df])


In [12]:
df = df_pro.reset_index(drop=True)

In [13]:
# Obtener la fecha actual
fecha_actual = datetime.today().strftime('%Y-%m-%d')  # Formato YYYY-MM-DD

# Agregar una nueva columna con la fecha actual al DataFrame
df['Fecha_actual'] = fecha_actual
df

,ID_cat,subcategoria,ID_subcat,categoria_producto,ID_pro,Nombre,Precio,Precio_unidad,Unidad_medida,Fecha_actual
0,112,"Aceite, vinagre y sal",420,Aceite de oliva,4240,"Aceite de oliva 0,4º Hacendado",8.00,8.00,l,2024-06-09
1,112,"Aceite, vinagre y sal",420,Aceite de oliva,4717,Aceite de oliva virgen extra Hacendado,28.80,9.60,l,2024-06-09
2,112,"Aceite, vinagre y sal",420,Aceite de oliva,4740,Aceite de oliva virgen extra Hacendado,9.65,9.65,l,2024-06-09
3,112,"Aceite, vinagre y sal",420,Aceite de oliva,4706,Aceite de oliva virgen extra Hacendado Gran Se...,8.10,10.80,l,2024-06-09
4,112,"Aceite, vinagre y sal",420,Aceite de oliva,4640,Aceite de oliva 1º Hacendado,8.00,8.00,l,2024-06-09
...,...,...,...,...,...,...,...,...,...,...
4978,98,Tomate y otros sabores,101,Otros sabores,39643,Néctar de maracuyá Hacendado fruta de la pasió...,1.30,1.30,l,2024-06-09
4979,98,Tomate y otros sabores,101,Otros sabores,60588,Bebida de mango Hacendado sin azúcares añadidos,1.25,1.25,l,2024-06-09
4980,98,Tomate y otros sabores,101,Otros sabores,39604,Bebida de pomelo Hacendado sin azúcares añadidos,1.25,1.25,l,2024-06-09
4981,98,Tomate y otros sabores,101,Otros sabores,39922,Zumo de manzana selección Hacendado,1.50,1.50,l,2024-06-09


In [14]:
Marcas_blancas= ['Hacendado','Bosque Verde','Deliplus','Compy']
def buscar_marca(texto, palabras):
    for palabra in palabras:
        if palabra in str(texto):
            return True
    return False

df['Marca'] = df['Nombre'].apply(lambda x: 'Marca blanca' if buscar_marca(x, Marcas_blancas) else 'Otras marcas')
df

,ID_cat,subcategoria,ID_subcat,categoria_producto,ID_pro,Nombre,Precio,Precio_unidad,Unidad_medida,Fecha_actual,Marca
0,112,"Aceite, vinagre y sal",420,Aceite de oliva,4240,"Aceite de oliva 0,4º Hacendado",8.00,8.00,l,2024-06-09,Marca blanca
1,112,"Aceite, vinagre y sal",420,Aceite de oliva,4717,Aceite de oliva virgen extra Hacendado,28.80,9.60,l,2024-06-09,Marca blanca
2,112,"Aceite, vinagre y sal",420,Aceite de oliva,4740,Aceite de oliva virgen extra Hacendado,9.65,9.65,l,2024-06-09,Marca blanca
3,112,"Aceite, vinagre y sal",420,Aceite de oliva,4706,Aceite de oliva virgen extra Hacendado Gran Se...,8.10,10.80,l,2024-06-09,Marca blanca
4,112,"Aceite, vinagre y sal",420,Aceite de oliva,4640,Aceite de oliva 1º Hacendado,8.00,8.00,l,2024-06-09,Marca blanca
...,...,...,...,...,...,...,...,...,...,...,...
4978,98,Tomate y otros sabores,101,Otros sabores,39643,Néctar de maracuyá Hacendado fruta de la pasió...,1.30,1.30,l,2024-06-09,Marca blanca
4979,98,Tomate y otros sabores,101,Otros sabores,60588,Bebida de mango Hacendado sin azúcares añadidos,1.25,1.25,l,2024-06-09,Marca blanca
4980,98,Tomate y otros sabores,101,Otros sabores,39604,Bebida de pomelo Hacendado sin azúcares añadidos,1.25,1.25,l,2024-06-09,Marca blanca
4981,98,Tomate y otros sabores,101,Otros sabores,39922,Zumo de manzana selección Hacendado,1.50,1.50,l,2024-06-09,Marca blanca


In [15]:
nombre_buscado = 'Desodorante aqua Deliplus antimanchas'

fila = df.loc[df['Nombre'] == nombre_buscado]

if not fila.empty:
    print("Se encontró el nombre en el DataFrame:")
    print(fila)
else:
    print("El nombre no se encontró en el DataFrame.")

Se encontró el nombre en el DataFrame:
      ID_cat subcategoria  ID_subcat categoria_producto ID_pro  \
2616     188  Desodorante        607  Desodorante Spray  23515   

                                     Nombre Precio Precio_unidad  \
2616  Desodorante aqua Deliplus antimanchas   1.70          8.50   

     Unidad_medida Fecha_actual         Marca  
2616             l   2024-06-09  Marca blanca  


In [16]:
df.to_csv(r"Mercadona.csv",index=False)

In [17]:
#Realizamos un EDA
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd